06.09.16

In [1]:
import numpy as np
hprint = lambda arg: pretty_print(html(arg))

<h2>Data input</h2>

In [10]:
size = 3
A = matrix(RDF, [[4.05, -3.5,  -1,    2.74,  0],
                [0,    2.74,  -2.25, 2.25,  1],
                [0,    0,     4.33,  3.45,  -3.45],
                [2.12, 1,     2.57,  -4.33, 0.29],
                [2.64, -1.73, 0,     2.38,  -3.04]])
#A = matrix(QQ, size, size, list(np.random.randint(size^2, size=size^2)))
#A = matrix(QQ, [[2, 3, 1], [4, 6, 7], [5, 4, 1]])
#A = matrix(QQ, [[0, 3, 1], [0, 6, 2], [0, 4, 1]])
b = vector(RDF, [6.27, 12.48, 12.92, -12.02, 12.4])
#b = vector(QQ, np.random.randint(size, size=size))
show('\mathbf{A}=' + latex(A))
# hprint('$\mathbf{A}=$')
# show(A)
show('\mathbf{b}=' + latex(b))

\mathbf{A}= \left(\begin{array}{rrrrr}
4.05 & -3.5 & -1.0 & 2.74 & 0.0 \\
0.0 & 2.74 & -2.25 & 2.25 & 1.0 \\
0.0 & 0.0 & 4.33 & 3.45 & -3.45 \\
2.12 & 1.0 & 2.57 & -4.33 & 0.29 \\
2.64 & -1.73 & 0.0 & 2.38 & -3.04
\end{array}\right)

\mathbf{b}= \left(6.27,\,12.48,\,12.92,\,-12.02,\,12.4\right)

<h2>Forward Elimination</h2>

In [11]:
control = sum(A.columns()) + b
show(r'\hat{\mathbf{b}}=' + latex(control))
B = A.augment(b).augment(control)
show('\mathbf{A}_\mathrm{augmented}\ \ =' + latex(B))

var_order = list(range(B.nrows()))

\hat{\mathbf{b}}= \left(8.56,\,16.22,\,17.25,\,-10.37,\,12.65\right)

\mathbf{A}_\mathrm{augmented}\ \ = \left(\begin{array}{rrrrrrr}
4.05 & -3.5 & -1.0 & 2.74 & 0.0 & 6.27 & 8.56 \\
0.0 & 2.74 & -2.25 & 2.25 & 1.0 & 12.48 & 16.22 \\
0.0 & 0.0 & 4.33 & 3.45 & -3.45 & 12.92 & 17.25 \\
2.12 & 1.0 & 2.57 & -4.33 & 0.29 & -12.02 & -10.37 \\
2.64 & -1.73 & 0.0 & 2.38 & -3.04 & 12.4 & 12.65
\end{array}\right)

In [12]:
def reduce_submatrix(B, offset):
    print('Normalizing...')
    leading_div = B[offset, offset]
    for i in range(B.ncols() - offset):
        print('%s /= %s' % (B[offset, offset + i], leading_div))
        B[offset, offset + i] /= leading_div
    show(B)
    
    print('Adding...')
    for i in range(1, B.nrows() - offset):
        leading_mult = B[offset + i, offset]
        for j in range(B.ncols() - offset):
            print('%s -= %s * %s' % (B[offset + i, offset + j], leading_mult, B[offset, offset + j]))
            B[offset + i, offset + j] -= leading_mult * B[offset, offset + j]
    show(B)

def swap_with_nonzero_lead(B, row_num):
    for i in [row_num..B.nrows()-1]:
        if B[i, row_num] != 0:
            print('Swapping %s with %s!' % (row_num, i))
            B.swap_rows(row_num, i)
            show(B)
            return True
    return False


def reduced_row_echelon_form(B, pivot_selection=False, with_control=True):
    for row_num in [0..B.nrows()-1]:
        print('Row %s' % row_num)
        show(B)
        if pivot_selection:
            print('Selecting pivot...')
            pivot = B[row_num, row_num]
            pivot_row = pivot_col = row_num
            for i in [row_num+1..B.nrows()-1]:
                for j in [row_num+1..B.ncols()-2-with_control]:
                    if abs(B[i, j]) > abs(pivot):
                        pivot = B[i, j]
                        pivot_row = i
                        pivot_col = j
            B.swap_rows(row_num, pivot_row)
            B.swap_columns(row_num, pivot_col)
            temp = var_order[pivot_col]
            var_order[pivot_col] = var_order[row_num]
            var_order[row_num] = temp
            show(B)
        if B[row_num, row_num] == 0:
            if pivot_selection or not swap_with_nonzero_lead(B, row_num):
                print("Column of all zeros; this isn't supported.")
                return False
        reduce_submatrix(B, row_num)
    return var_order

In [13]:
hprint('<!--notruncate-->')
print(reduced_row_echelon_form(B, pivot_selection=True))
show(B)

<!--notruncate-->

Row 0


[               4.05                -3.5                -1.0                2.74                 0.0                6.27   8.559999999999999]
[                0.0                2.74               -2.25                2.25                 1.0               12.48               16.22]
[                0.0                 0.0                4.33                3.45               -3.45               12.92               17.25]
[               2.12                 1.0                2.57               -4.33                0.29              -12.02 -10.370000000000001]
[               2.64               -1.73                 0.0                2.38               -3.04                12.4               12.65]

Normalizing...
4.05 /= 4.05
-3.5 /= 4.05
-1.0 /= 4.05
2.74 /= 4.05
0.0 /= 4.05
6.27 /= 4.05
8.56 /= 4.05


[                1.0 -0.8641975308641976 -0.2469135802469136  0.6765432098765433                 0.0   1.548148148148148    2.11358024691358]
[                0.0                2.74               -2.25                2.25                 1.0               12.48               16.22]
[                0.0                 0.0                4.33                3.45               -3.45               12.92               17.25]
[               2.12                 1.0                2.57               -4.33                0.29              -12.02 -10.370000000000001]
[               2.64               -1.73                 0.0                2.38               -3.04                12.4               12.65]

...

[                 1.0  -0.8641975308641976  -0.2469135802469136   0.6765432098765433                  0.0    1.548148148148148     2.11358024691358]
[                 0.0                  1.0  -0.8211678832116788   0.8211678832116788  0.36496350364963503    4.554744525547445    5.919708029197079]
[                 0.0                  0.0                  1.0   0.7967667436489607  -0.7967667436489607    2.983833718244804    3.983833718244804]
[                 0.0                  0.0                  0.0                  1.0 -0.28805898914865524   3.5761179782973103    4.288058989148655]
[                 0.0                  0.0                  0.0                  0.0                  1.0  -2.0000000000000004  -1.0000000000000004]

Adding...


[                 1.0  -0.8641975308641976  -0.2469135802469136   0.6765432098765433                  0.0    1.548148148148148     2.11358024691358]
[                 0.0                  1.0  -0.8211678832116788   0.8211678832116788  0.36496350364963503    4.554744525547445    5.919708029197079]
[                 0.0                  0.0                  1.0   0.7967667436489607  -0.7967667436489607    2.983833718244804    3.983833718244804]
[                 0.0                  0.0                  0.0                  1.0 -0.28805898914865524   3.5761179782973103    4.288058989148655]
[                 0.0                  0.0                  0.0                  0.0                  1.0  -2.0000000000000004  -1.0000000000000004]

[0, 1, 2, 3, 4]


[                 1.0  -0.8641975308641976  -0.2469135802469136   0.6765432098765433                  0.0    1.548148148148148     2.11358024691358]
[                 0.0                  1.0  -0.8211678832116788   0.8211678832116788  0.36496350364963503    4.554744525547445    5.919708029197079]
[                 0.0                  0.0                  1.0   0.7967667436489607  -0.7967667436489607    2.983833718244804    3.983833718244804]
[                 0.0                  0.0                  0.0                  1.0 -0.28805898914865524   3.5761179782973103    4.288058989148655]
[                 0.0                  0.0                  0.0                  0.0                  1.0  -2.0000000000000004  -1.0000000000000004]

<h3>Comparing with control</h3>

In [14]:
def is_close(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

for i in [0..B.nrows()-1]:
    B_mod_sum_i = sum(B[i][:-1])
    control_mod_i = B[i, -1]
    if not is_close(B_mod_sum_i, control_mod_i):
        print('%s != %s' % (B_mod_sum_i, control_mod_i))
        print('Check failed at row %s' % i)
        break
    else:
        print('%s == %s' % (B_mod_sum_i, control_mod_i))
    if i == B.nrows()-1:
        print('All checks passed.')

2.11358024691 == 2.11358024691
5.9197080292 == 5.9197080292
3.98383371824 == 3.98383371824
4.28805898915 == 4.28805898915
-1.0 == -1.0
All checks passed.


<h2>Back Substitution</h2>

In [15]:
def sort_by_list(v, idcs):
    return vector(RDF, (elem for (idx, elem) in sorted(enumerate(v), key=lambda (idx, elem): idcs[idx])))

In [16]:
def back_substitution(A_aug):
    N = A_aug.nrows()
    b_mod = A_aug.columns()[N]
    x_star = vector(RDF, [0 for _ in range(N)])
    for i in reversed(range(N)):
        x_star[i] = b_mod[i] - sum(A_aug[i, k] * x_star[k] for k in range(i+1, N))
    return sort_by_list(x_star, var_order)

x_star = back_substitution(B)
show('\mathbf{x}^*=' + latex(x_star))

\mathbf{x}^*= \left(1.0,\,2.0,\,-1.0,\,3.0,\,-2.0\right)

<h2>Testing solution</h2>

In [18]:
show('\mathbf{A}\mathbf{x}^*=' + latex(A * x_star))
show('\mathbf{b}=' + latex(b))
show('\mathbf{A}\mathbf{x}^* - \mathbf{b}=' + latex(A * x_star - b))

\mathbf{A}\mathbf{x}^*= \left(6.27,\,12.48,\,12.92,\,-12.02,\,12.4\right)

\mathbf{b}= \left(6.27,\,12.48,\,12.92,\,-12.02,\,12.4\right)

\mathbf{A}\mathbf{x}^* - \mathbf{b}= \left(-8.881784197 \times 10^{-16},\,0.0,\,1.7763568394 \times 10^{-15},\,1.7763568394 \times 10^{-15},\,1.7763568394 \times 10^{-15}\right)